### Please Install Plotly on your machine to see plots in notebooks.

# <u> Spectral Clustering </u>
reference: https://www.kaggle.com/vipulgandhi/spectral-clustering-detailed-explanation

In spectral clustering, data points are treated as nodes of a graph. Thus, spectral clustering is a graph partitioning problem. The nodes are then mapped to a low-dimensional space that can be easily segregated to form clusters. No assumption is made about the shape/form of the clusters. The goal of spectral clustering is to cluster data that is connected but not necessarily compact or clustered within convex boundaries.

### <u> Spectral Clustering vs. Kmeans </u>

- <b><u> Compactness </u></b> — Points that lie close to each other fall in the same cluster and are compact around the cluster center. The closeness can be measured by the distance between the observations. E.g.: <b> K-Means Clustering </b>
- <b><u> Connectivity </u></b> — Points that are connected or immediately next to each other are put in the same cluster. Even if the distance between 2 points is less, if they are not connected, they are not clustered together. <b> Spectral Clustering </b> is a technique that follows this approach.
 
 K-means will fail to effectively cluster these, even when the true number of clusters K is known to the algorithm. K-means, as a <i> data-clustering </i> algorithm, ideal for discovering globular clusters where all members of each cluster are in close proximity to each other (in Euclidean sense).

Spectral clustering is more general (and powerful) because if we just use Eucledean Distance in it's similarity matrix, It will behave like k-means. The converse is not true though.

If we have 𝑃 data points each with 𝑁 dimensions/features, input matrix to K-means would be 𝑁 by 𝑃, while input matrix to spectral clustering would be 𝑃 by 𝑃. Spectral clustering is indifferent to the number of features we use (Gaussian kernel which can be thought of as an infinite-dimensional feature transformation is particularly popular when using spectral clustering). We will face difficulties applying spectral clustering (at least the vanilla version) to very large datasets (large 𝑃).

### <u> Algorithm: </u>
- Project data into $R^{n}$ matrix
- Define an Affinity  matrix A , using a Gaussian Kernel K  or an Adjacency matrix
- Construct the Graph Laplacian from A  (i.e. decide on a normalization)
- Solve the Eigenvalue problem
- Select k eigenvectors corresponding to the k lowest (or highest) eigenvalues to define a k-dimensional subspace 
- Form clusters in this subspace using k-means

#### <u> Affinity Matrix: </u>

We first create an undirected graph G = (V, E) with vertex set V = {v1, v2, …, vn} = 1, 2, …, n observations in the data.

- <b> <u> Epsilon-neighbourhood Graph: </u> </b> A parameter epsilon is fixed beforehand. Then each point is connected to all the points which lie in it’s epsilon-radius. If all the distances between any two points are similar in scale then typically the weights of the edges ie the distance between the two points are not stored since they do not provide any additional information. Thus, in this case, the graph built is an undirected and unweighted graph.
- <b> <u> K-Nearest Neighbours: </u> </b> A parameter k is fixed beforehand. Then, for two vertices u and v, an edge is directed from u to v only if v is among the k-nearest neighbours of u. Note that this leads to the formation of a weighted and directed graph because it is not always the case that for each u having v as one of the k-nearest neighbours, it will be the same case for v having u among its k-nearest neighbours. To make this graph undirected, one of the following approaches are followed:-
     - Direct an edge from u to v and from v to u if either v is among the k-nearest neighbours of u <b> OR </b> u is among the k-nearest neighbours of v.
     - Direct an edge from u to v and from v to u if v is among the k-nearest neighbours of u <b> AND </b> u is among the k-nearest neighbours of v.
     
- <b> <u> Fully-Connected Graph: </u></b> To build this graph, each point is connected with an undirected edge-weighted by the distance between the two points to every other point. Since this approach is used to model the local neighbourhood relationships thus typically the Gaussian similarity metric is used to calculate the distance.

$$S(x_i, x_j) = exp(-\frac{||x_i - x_j||^2}{2\sigma^2})$$

Thus, when we create an adjacency matrix for any of these graphs, Aij ~ 1 when the points are close and Aij → 0 if the points are far apart. 

Consider the following graph with nodes 1 to 4, weights (or similarity) wij and its adjacency matrix:


<b><u>Metric :</u></b>

 Affinity metric determines how close, or similar, two points our in our space. We will use a Gaussian Kernel and not the standard Euclidean metric.

Given 2 data points $x_{i},x_{j}$  (projected in $R^{n}$ ), we define an Affinity $A_{i,j}$  that is positive, symmetric, and depends on the Euclidian distance $\Vert x_{i}-x_{j}\Vert$  between the data points

$$A_{ij} = {e}^{-\alpha \Vert x_{i}-x_{j}\Vert^2 }$$

We might provide a hard cut off R , so that

$A_{ij} = 0$  if $\Vert x_{i}-x_{j}\Vert^2 \geq R$

$A_{i,j}\simeq 1$  when the points are close in $R^{n}$ , and $A_{i,j}\rightarrow 0$  if the points $x_{i}$, $x_{j}$ are far apart. Close data points are in the same cluster. Data points in different clusters are far away. But data points in the same cluster may also be far away–even farther away than points in different clusters. Our goal then is to transform the space so that when 2 points $x_{i}$, $x_{j}$ are close, they are always in same cluster, and when they are far apart, they are in different clusters. Generally we use the Gaussian Kernel K  directly, or we form the Graph Laplacian A.

<b><u>Graph Laplacian</u></b> is just another matrix representation of a graph. It can be computed as:

- Simple Laplacian $L=D-A$
- Normalized Laplacian $L_{N}=D^{-1/2}LD^{-1/2}$
- Generalized Laplacian $L_{G} = D^{-1}L$
- Relaxed Laplacian $L_{\rho} = L-\rho D $
- Ng, Jordan, & Weiss Laplacian $L_{NJW}=D^{-1/2}AD^{-1/2}, where A_{i,i}=0 $

$L = D - A$ where A is the Adjacency matrix and D is the Degree Matrix.

$$D_i = \sum_{j|(i,j) \in E} w_{ij}$$
$$ L_{ij} =
  \begin{cases}
    d_i           & \quad \text{if } i = j\\
    -w_{ij}       & \quad \text{if } i , j \in E \\
    0             & \quad \text{if } i , j \notin  E
  \end{cases}
$$


The whole purpose of computing the Graph Laplacian L was to find eigenvalues and eigenvectors for it, in order to embed the data points into a low-dimensional space.


<b><u>The Cluster Eigenspace Problem</u></b>

To identify good clusters, Laplacian L should be approximately a block-diagonal, with each block defining a cluster. If we have 3 major clusters (C1, C2, C3), we would expect

$$\begin{matrix}
  L_{1,1} & L_{1,2} & L_{1,3} \\
  L_{2,1} & L_{2,2} & L_{2,3} \\
  L_{3,1} & L_{3,2} & L_{3,3}
 \end{matrix}
 \sim
 \begin{matrix}
  L_{1,1} & 0 & 0 \\
  0 & L_{2,2} & 0 \\
  0 & 0 & L_{3,3}
 \end{matrix}$$

 
 We also expect that the 3 lowest eigenvalues  & eigenvectors (\lambda_{i},v_{i})  of L  each correspond to a different cluster.
 
 For K clusters, compute the first K eigen vectors. ${v_1, v_2, ...v_k}$. Stack the vectors vertically to form the matrix with eigen vecttors as columns. Represent every node as the corresponding row of this new matrix, these rows form the feature vector of the nodes. Use Kmeans to cluster these points into k clusters $C_1, C_2 ...C_k$
 

reference: https://www.kaggle.com/vipulgandhi/spectral-clustering-detailed-explanation

In [1]:
import functools
import numpy as np
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from mpl_toolkits.mplot3d import Axes3D
from sklearn.neighbors import kneighbors_graph
from sklearn.cluster import SpectralClustering
from sklearn.preprocessing import StandardScaler, normalize 

In [2]:
data = pd.read_csv("fraud_cluster.csv")
data.head()

,P1,P2,P3,P4,P5,P6,P7,P8,P9
0,0.595378,-0.531958,0.679654,-0.126799,0.432046,0.988092,-0.029813,0.768742,-0.054167
1,0.982237,0.991481,0.337646,0.228144,0.920032,0.999985,-0.032259,2.161651,-0.054350
2,0.996162,0.893987,0.767413,0.606840,0.970808,0.882602,-0.032267,0.369607,-0.054157
3,0.999928,0.922748,-0.444438,-0.371287,0.528038,-0.221645,-0.032692,-1.065439,0.381914
4,0.985838,0.937512,0.699592,0.585263,0.838804,0.999602,-0.033713,2.089720,-0.054379


In [3]:
data.shape

(1163, 9)

In [4]:
X = data.values
X.shape

(1163, 9)

In [5]:
tsne_model = TSNE(perplexity=40, n_components=3, init='pca', n_iter=500, random_state=23)
new_values = tsne_model.fit_transform(X)

In [6]:
def plot_data(new_values, labels):
    output = pd.DataFrame(new_values, columns=['x', 'y', 'z'])
    output['class'] = labels
    fig = px.scatter_3d(output, x='x', y='y', z='z', opacity=1.0, color='class')
    fig.update_traces(marker=dict(size=2), selector=dict(mode='markers'))
    fig.update_layout(margin={'l': 0, 'r': 0, 'b': 0, 't': 0}, width=600, height=400)
    fig.show()

In [7]:
# scale the data
scaler = StandardScaler() 
X_scaled = scaler.fit_transform(X) 
  
# Normalizing the Data 
X = normalize(X_scaled)
X.shape

(1163, 9)

In [8]:
A = np.zeros((X.shape[0], X.shape[0]))
for i in range(X.shape[0]):
    for j in range(X.shape[0]):
        A[i][j] = -3 * np.sum((X[i] - X[j])**2)
A = np.exp(A)
print(A)

[[1.00000000e+00 1.91956382e-03 2.33766140e-03 ... 4.16654452e-03
  7.82883446e-05 7.18617192e-03]
 [1.91956382e-03 1.00000000e+00 8.22039363e-02 ... 2.32668595e-03
  2.93557757e-05 1.39894781e-04]
 [2.33766140e-03 8.22039363e-02 1.00000000e+00 ... 9.24043303e-04
  3.16143965e-04 2.04085249e-04]
 ...
 [4.16654452e-03 2.32668595e-03 9.24043303e-04 ... 1.00000000e+00
  6.54234414e-04 7.10718881e-02]
 [7.82883446e-05 2.93557757e-05 3.16143965e-04 ... 6.54234414e-04
  1.00000000e+00 1.37210265e-02]
 [7.18617192e-03 1.39894781e-04 2.04085249e-04 ... 7.10718881e-02
  1.37210265e-02 1.00000000e+00]]


In [9]:
D = np.diag(A.sum(axis=1))

## Simple Laplacian L = D - A

In [10]:
L = D - A

In [11]:
vals, vecs = np.linalg.eig(L)

# sort
vecs = vecs[:,np.argsort(vals)]

In [12]:
feats = vecs[:, :2]
feats.shape

(1163, 2)

In [13]:
kmeans = KMeans(n_clusters=4, random_state=42).fit(feats)
print(np.asarray(np.unique(kmeans.labels_, return_counts=True)).T)

[[   0   37]
 [   1    1]
 [   2 1124]
 [   3    1]]


In [14]:
plot_data(new_values, kmeans.labels_)

## Trying different Laplacian

In [15]:
L = D - A
Dinv = 1 / (np.linalg.inv(D)**(1/2) + 0.00000001)
Lnew = Dinv * L * Dinv

In [16]:
vals, vecs = np.linalg.eig(Lnew)

# sort
vecs = vecs[:,np.argsort(vals)]

In [17]:
feats = vecs[:, :4]
feats.shape

(1163, 4)

In [18]:
kmeans = KMeans(n_clusters=5, random_state=42).fit(feats)
print(np.asarray(np.unique(kmeans.labels_, return_counts=True)).T)

[[  0 457]
 [  1 171]
 [  2 198]
 [  3 175]
 [  4 162]]


In [19]:
plot_data(new_values, kmeans.labels_)

## Trying different Affinity Matrix

In [20]:
A = kneighbors_graph(X, n_neighbors=5).toarray()
print(A)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [21]:
D = np.diag(A.sum(axis=1))

In [22]:
L = D - A

In [23]:
vals, vecs = np.linalg.eig(L)

# sort
vecs = vecs[:,np.argsort(vals)]

In [24]:
feats = vecs[:, :2].real
feats.shape

(1163, 2)

In [25]:
kmeans = KMeans(n_clusters=4, random_state=42).fit(feats)
print(np.asarray(np.unique(kmeans.labels_, return_counts=True)).T)

[[  0 339]
 [  1 336]
 [  2 211]
 [  3 277]]


In [26]:
plot_data(new_values, kmeans.labels_)

## Best Model for Fraud Detection

In [27]:
A = np.zeros((X.shape[0], X.shape[0]))
for i in range(X.shape[0]):
    for j in range(X.shape[0]):
        A[i][j] = -3 * np.sum((X[i] - X[j])**2)
A = np.exp(A)
print(A)

[[1.00000000e+00 1.91956382e-03 2.33766140e-03 ... 4.16654452e-03
  7.82883446e-05 7.18617192e-03]
 [1.91956382e-03 1.00000000e+00 8.22039363e-02 ... 2.32668595e-03
  2.93557757e-05 1.39894781e-04]
 [2.33766140e-03 8.22039363e-02 1.00000000e+00 ... 9.24043303e-04
  3.16143965e-04 2.04085249e-04]
 ...
 [4.16654452e-03 2.32668595e-03 9.24043303e-04 ... 1.00000000e+00
  6.54234414e-04 7.10718881e-02]
 [7.82883446e-05 2.93557757e-05 3.16143965e-04 ... 6.54234414e-04
  1.00000000e+00 1.37210265e-02]
 [7.18617192e-03 1.39894781e-04 2.04085249e-04 ... 7.10718881e-02
  1.37210265e-02 1.00000000e+00]]


In [28]:
D = np.diag(A.sum(axis=1))

In [29]:
L = D - A

In [30]:
vals, vecs = np.linalg.eig(L)

# sort
vecs = vecs[:,np.argsort(vals)]

In [31]:
feats = vecs[:, :2]
feats.shape

(1163, 2)

In [32]:
kmeans = KMeans(n_clusters=4, random_state=42).fit(feats)
print(np.asarray(np.unique(kmeans.labels_, return_counts=True)).T)

[[   0   37]
 [   1    1]
 [   2 1124]
 [   3    1]]


In [33]:
plot_data(new_values, kmeans.labels_)